In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize, LabelBinarizer
import sys
sys.path.append('../..')
from modules.many_features import utils, constants

In [2]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

#### The Datasets

In [59]:
train_df = pd.read_csv('../../final/data/train_set_0.05')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,6.104489,-1.000000,-1.000000,0.920616,150.863286,77.936869,45.018443,2.349782,1,1.297655,34.961997,60.378870,7.798268,6.015069,112.750517,18.313467,29.840555
1,7.781663,215.967826,1.643871,5.956930,-1.000000,85.512847,-1.000000,2.729998,0,1.157021,87.155613,-1.000000,-1.000000,-1.000000,-1.000000,23.344989,-1.000000
2,10.306951,-1.000000,-1.000000,3.269642,361.637557,101.949912,213.802158,3.032946,0,1.983531,3.371683,128.512264,14.157427,19.486851,133.035690,30.920853,59.120563
3,8.034653,-1.000000,5.322562,1.290493,-1.000000,89.288985,-1.000000,2.699544,0,1.135298,57.357480,73.669678,-1.000000,1.485512,-1.000000,24.103959,-1.000000
4,8.314152,-1.000000,0.222138,6.084132,-1.000000,81.319810,-1.000000,3.067205,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,24.942456,-1.000000


In [60]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000


In [61]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2800, 17), (14000, 17), (2800,), (14000,))

In [62]:
y_test

array([5, 1, 4, ..., 2, 4, 6])

#### Normalizing X

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [64]:
mmc = MinMaxScaler()

In [65]:
X_train_norm = mmc.fit_transform(X_train)
X_test_norm = mmc.transform(X_test)

In [66]:
X_train_norm.shape, X_test_norm.shape

((2800, 17), (14000, 17))

#### Support Vector Machine

In [67]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=constants.SEED).fit(X_train_norm, y_train)
y_pred_svm = svm_model.predict(X_test_norm)
test_df_svm = pd.DataFrame()
test_df_svm['y_actual'] = y_test
test_df_svm['y_pred'] = y_pred_svm
test_df_svm.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [68]:
success_rate_svm, success_df_svm = utils.success_rate(test_df_svm)
success_rate_svm

75.91428571428571

In [69]:
acc_svm, f1_svm, roc_auc_svm = utils.test(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm, f1_svm, roc_auc_svm

(0.7591428571428571, 0.7258051390121358, 0.8523933709329672)

#### Feed-forward Neural Network - Keras

In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
import keras

2022-11-28 01:54:19.556009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 01:54:21.450152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-28 01:54:21.450305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-28 01:54:21.450316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [17]:
#keras.utils.set_random_seed(42)

In [18]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
def numerize_labels(pd_series):
    series_copy = pd_series.copy()
    series_copy = series_copy.map(class_dict)
    return series_copy

y_train_num = y_train
y_test_num = y_test

dummy_y_train = np_utils.to_categorical(y_train_num)
dummy_y_test = np_utils.to_categorical(y_test_num)
dummy_y_train.shape, dummy_y_test.shape

((56000, 8), (14000, 8))

In [20]:
model = Sequential()
model.add(Dropout(0.5, input_shape=(17,)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
history = model.fit(X_train_norm, dummy_y_train, epochs=100, batch_size=1024, shuffle=True, validation_split=0.20, verbose=0)
#model.save('../models/baselines/synthentic_nn.h5')
loss, accuracy, f1, precision, recall = model.evaluate(X_test_norm, dummy_y_test, verbose=0)
loss, accuracy, f1, precision, recall

2022-11-24 20:01:27.447634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-24 20:01:27.447667: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-24 20:01:27.447687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pl823-pro.paris.inria.fr): /proc/driver/nvidia/version does not exist
2022-11-24 20:01:27.447896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(1.9023798704147339, 0.267214298248291, 0.0, 0.0, 0.0)

In [21]:
#Alternatively
y_pred1_nn = model.predict(X_test_norm)
y_pred_nn = np.argmax(y_pred1_nn, axis=1)
np.unique(y_pred_nn)

438/438 [==============================] - 1s 2ms/step


array([0, 1, 2, 3, 6])

In [22]:
test_df_nn = pd.DataFrame()
test_df_nn['y_actual'] = y_test
test_df_nn['y_pred'] = y_pred_nn
test_df_nn.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [23]:
success_rate_nn, success_df_nn = utils.success_rate(test_df_nn)
success_rate_nn

26.721428571428575

In [24]:
acc_nn, f1_nn, roc_auc_nn = utils.test(test_df_nn['y_actual'], test_df_nn['y_pred'])
acc_nn, f1_nn, roc_auc_nn

(0.26721428571428574, 0.17606350707284107, 0.5727130314417213)